In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import math

In [2]:
snp = pd.read_csv('S&P500.csv',parse_dates=True, index_col='Date')
snp_ret = np.log(snp/snp.shift(1))
snp_ret

,Close
Date,
2016-11-15,NaN
2016-11-16,-0.001584
2016-11-17,0.004665
2016-11-18,-0.002390
2016-11-21,0.007434
...,...
2019-11-08,0.002557
2019-11-11,-0.001964
2019-11-12,0.001563


In [3]:
#1 S&P Annual Volatility
annual_vol = snp_ret.std() * np.sqrt(252)
annual_vol[0]

0.12831845548060564

# Black Scholes Merton Model

In [4]:
t = 1/12
so = snp.Close[-1]
k = 3100
q =.023
rf = 0.05
vol = annual_vol[0]



def blackscholes(option,so,k,rf,t,vol,div=0):
    d1 = (np.log(so/k)+(rf-div+vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = d1-vol*np.sqrt(t)
    if option == 'call':
        call = so*math.e**(-div*t) * norm.cdf(d1) - k*math.e**(-rf*t) * norm.cdf(d2)
        return call
    elif option == 'put':
        put = k*math.e**(-rf*t)*norm.cdf(-d2) - so*math.e**(-div*t)*norm.cdf(-d1)
        return put
    

In [5]:
cboe = blackscholes('call',so,k,rf,t,vol,q)
print(f'Value of CBOE Option: {cboe*100}')

Value of CBOE Option: 4746.202827704724


In [6]:
np.random.normal()
so
vol

0.12831845548060564

# Monte Carlo 

In [7]:
# so = 10
# k = 9.5
# vol = 0.2
# r = 0.05
# t=0.5


mc = pd.DataFrame({'counter': range(1,1001,1),'norm' : np.random.normal(size=(1000))})
mc

,counter,norm
0,1,-0.855885
1,2,0.283054
2,3,0.066299
3,4,-0.617798
4,5,0.002038
...,...,...
995,996,1.508324
996,997,-0.926196
997,998,-1.501258
998,999,0.547063


In [8]:
mc['Gross_return'] = math.e**((rf-q-.5*vol**2)*t + vol * np.sqrt(t)*mc.norm)

In [9]:
mc['Price'] = mc.Gross_return * so
mc['Payoff'] = np.where(mc.Price > k, mc.Price - k, 0)
mc['fi'] = math.e**(-rf*t) * mc.Payoff
mc.head()

,counter,norm,Gross_return,Price,Payoff,fi
0,1,-0.855885,0.970310,3004.689729,0.000000,0.000000
1,2,0.283054,1.012122,3134.166574,34.166574,34.024510
2,3,0.066299,1.004028,3109.102832,9.102832,9.064983
3,4,-0.617798,0.978905,3031.306188,0.000000,0.000000
4,5,0.002038,1.001641,3101.710734,1.710734,1.703621


In [10]:
std_error = mc.fi.std()/np.sqrt(mc.shape[0])
std_error

2.2302782686025115

In [11]:
pv_mean = mc.fi.mean()
print(f'PV Mean: {pv_mean}')
option_val = pv_mean * 100
print(f'Option Value: {option_val}')
ciu = option_val + (1.96 * std_error) * 100
cid = option_val - (1.96 * std_error) * 100
print(f'Confidence Interval: ({cid}, {ciu})')

PV Mean: 48.45203866369936
Option Value: 4845.203866369936
Confidence Interval: (4408.069325723844, 5282.338407016028)


# Asian European Option

In [12]:
# 3/21
asian = pd.DataFrame({'counter': range(1,1001)}).set_index('counter')
asian['n1'] = np.random.normal(size=(1000))
asian['g1'] = math.e**((rf-q-.5*vol**2)*(t/21) + vol * np.sqrt(t/21)*asian['n1'])
asian['p1'] = asian['g1'] * so
for i in range(2,22,1):
    asian[f'n{i}'] = np.random.normal(size=(1000))
    asian[f'g{i}'] = math.e**((rf-q-.5*vol**2)*(t/21) + vol * np.sqrt(t/21)*asian[f'n{i}'])
    asian[f'p{i}'] = asian[f'g{i}'] * asian[f'p{i-1}']
asian


,n1,g1,p1,n2,g2,p2,n3,g3,p3,n4,...,p18,n19,g19,p19,n20,g20,p20,n21,g21,p21
counter,,,,,,,,,,,,,,,,,,,,,
1,-0.155931,0.998815,3092.959568,0.602253,1.004955,3108.284879,-2.224166,0.982255,3053.128968,0.916683,...,3086.815708,-0.945916,0.992457,3063.531664,-0.521144,0.995870,3050.880673,-0.323535,0.997462,3043.138966
2,1.688583,1.013818,3139.420365,-1.784084,0.985756,3094.701230,0.887908,1.007278,3117.224643,0.482473,...,3126.975449,-0.767300,0.993891,3107.872411,1.191734,1.009755,3138.189359,-0.325426,0.997447,3130.178241
3,-0.043280,0.999725,3095.777268,0.527812,1.004350,3109.245061,1.149425,1.009410,3138.501868,0.800144,...,3199.002808,0.481765,1.003977,3211.723993,-1.496808,0.988047,3173.335319,0.611189,1.005027,3189.289248
4,0.959178,1.007859,3120.964805,-0.902482,0.992805,3098.510839,1.177205,1.009636,3128.369053,0.044424,...,3267.175725,0.858802,1.007041,3290.180252,-0.096471,0.999295,3287.860399,-0.540197,0.995717,3273.778792
5,0.419850,1.003474,3107.388403,0.129625,1.001123,3110.877700,-0.069278,0.999515,3109.367674,-0.356579,...,2942.901748,-0.216312,0.998327,2937.979335,0.558250,1.004598,2951.486732,1.735706,1.014205,2993.411561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.088637,1.000791,3099.080138,0.003242,1.000101,3099.392164,-0.234039,0.998184,3093.764647,-1.110198,...,3114.821597,0.448682,1.003708,3126.371854,-0.336947,0.997354,3118.100529,1.930929,1.015806,3167.386390
997,-0.136451,0.998972,3093.446641,-1.013150,0.991918,3068.444519,0.786019,1.006449,3088.232324,1.795956,...,3189.628608,0.579242,1.004768,3204.836744,0.551446,1.004542,3219.393957,0.331668,1.002759,3228.277050
998,-0.315632,0.997526,3088.969413,0.218671,1.001844,3094.664725,0.848930,1.006961,3116.205891,2.016175,...,3236.744568,-0.482051,0.996185,3224.397054,-0.599104,0.995243,3209.058859,-0.737176,0.994133,3190.231091


In [13]:
filter_col = [col for col in asian if col.startswith('p')]
p = asian[filter_col]
asian['Avg Price'] = p.mean(axis=1)
asian['Payoff'] = np.where(asian['Avg Price'] > k, asian['Avg Price'] - k,0)
asian['PV'] = math.e**(-rf*t) * asian.Payoff
asian


,n1,g1,p1,n2,g2,p2,n3,g3,p3,n4,...,p19,n20,g20,p20,n21,g21,p21,Avg Price,Payoff,PV
counter,,,,,,,,,,,,,,,,,,,,,
1,-0.155931,0.998815,3092.959568,0.602253,1.004955,3108.284879,-2.224166,0.982255,3053.128968,0.916683,...,3063.531664,-0.521144,0.995870,3050.880673,-0.323535,0.997462,3043.138966,3091.025764,0.000000,0.000000
2,1.688583,1.013818,3139.420365,-1.784084,0.985756,3094.701230,0.887908,1.007278,3117.224643,0.482473,...,3107.872411,1.191734,1.009755,3138.189359,-0.325426,0.997447,3130.178241,3141.504007,41.504007,41.331433
3,-0.043280,0.999725,3095.777268,0.527812,1.004350,3109.245061,1.149425,1.009410,3138.501868,0.800144,...,3211.723993,-1.496808,0.988047,3173.335319,0.611189,1.005027,3189.289248,3157.244715,57.244715,57.006692
4,0.959178,1.007859,3120.964805,-0.902482,0.992805,3098.510839,1.177205,1.009636,3128.369053,0.044424,...,3290.180252,-0.096471,0.999295,3287.860399,-0.540197,0.995717,3273.778792,3153.087211,53.087211,52.866474
5,0.419850,1.003474,3107.388403,0.129625,1.001123,3110.877700,-0.069278,0.999515,3109.367674,-0.356579,...,2937.979335,0.558250,1.004598,2951.486732,1.735706,1.014205,2993.411561,3033.901640,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.088637,1.000791,3099.080138,0.003242,1.000101,3099.392164,-0.234039,0.998184,3093.764647,-1.110198,...,3126.371854,-0.336947,0.997354,3118.100529,1.930929,1.015806,3167.386390,3123.710061,23.710061,23.611474
997,-0.136451,0.998972,3093.446641,-1.013150,0.991918,3068.444519,0.786019,1.006449,3088.232324,1.795956,...,3204.836744,0.551446,1.004542,3219.393957,0.331668,1.002759,3228.277050,3162.021644,62.021644,61.763758
998,-0.315632,0.997526,3088.969413,0.218671,1.001844,3094.664725,0.848930,1.006961,3116.205891,2.016175,...,3224.397054,-0.599104,0.995243,3209.058859,-0.737176,0.994133,3190.231091,3208.096541,108.096541,107.647076


In [14]:
pv_mean = asian.PV.mean()
print(f'PV Mean: {pv_mean}]')
std_error_a = asian.PV.std()/np.sqrt(asian.shape[0])
opt_val = pv_mean * 100
print(f'Option Value: {opt_val}')
ciua = opt_val + (1.96 * std_error_a) * 100
cida = opt_val - (1.96 * std_error_a) * 100
print(f'Confidence Interval: ({cida}, {ciua})')


PV Mean: 28.559558359294]
Option Value: 2855.9558359294
Confidence Interval: (2607.772445559402, 3104.139226299398)


# Binomial Tree

In [15]:
# Binomial Tree
up = math.e**(vol*np.sqrt(t/3))
down = 1/up
so
pu = (math.e**((rf-q)*t/3) - down)/(up - down)
pd = (1-pu)
vu = so * up
vd = so * down
vuu = vu * up
vud = vu * down
vdd = vd * down

vuuu = vuu * up
vuud = vuu * down
vudd = vud * down
vddd = vdd * down

ruuu = vuuu - k
ruud = vuud - k

puuu = pu * pu * pu
puud = 3*pu*pu*pd

In [16]:
print(vuuu, vuud, vudd, vddd)
v1 = ruuu * puuu
v2 = ruud * puud
f1 = v1 + v2
value_call = f1* math.e ** (-rf*t)
print(f'Value of Call: {value_call}')
print(f'Value of Option: {value_call*100}')

3301.819279412705 3163.5689184130492 3031.107233503484 2904.1918472273887
Value of Call: 51.3093483606731
Value of Option: 5130.93483606731


In [19]:
norm.ppf(.95)

1.6448536269514722